In [1]:
import zipfile
import xml.etree.ElementTree as ET

# 엑셀 파일 경로
excel_file = './data/raw/CO_IMAGE_DSS_반출.xlsx'

# 이미지 순서와 파일명을 담을 리스트
image_order = []

# ZIP 파일 열기
with zipfile.ZipFile(excel_file, 'r') as z:
    # drawings 폴더의 XML 파일 확인
    drawing_files = [f for f in z.namelist() if f.startswith('xl/drawings/drawing')]
    
    for drawing_file in drawing_files:
        # drawing XML 파일 파싱
        with z.open(drawing_file) as file:
            tree = ET.parse(file)
            root = tree.getroot()

            # XML에서 이미지 ID 및 타겟 경로 추출
            namespaces = {'a': 'http://schemas.openxmlformats.org/drawingml/2006/main'}
            for blip in root.findall('.//a:blip', namespaces):
                # 이미지 파일 이름 추출 (e.g., image1.png)
                image_id = blip.attrib.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed')
                if image_id:
                    # Relationships 파일 경로 구성
                    rels_path = drawing_file.replace('drawings', 'drawings/_rels') + '.rels'
                    
                    # Relationship 파일에서 실제 이미지 파일 경로 추출
                    with z.open(rels_path) as rel_file:
                        rel_tree = ET.parse(rel_file)
                        rel_root = rel_tree.getroot()
                        for rel in rel_root:
                            if rel.attrib.get('Id') == image_id:
                                image_target = rel.attrib['Target']
                                if image_target.startswith('../media/'):
                                    image_name = image_target.split('/')[-1]
                                    image_order.append(image_name)


In [2]:
import zipfile
import os
import pandas as pd

def extract_images_with_ids(excel_file, output_folder):
    df = pd.read_excel(excel_file, header=None)

    with zipfile.ZipFile(excel_file, 'r') as z:
        image_files = ['xl/media/' + x for x in image_order]
        image_file_index = 0
        for i in range(1, len(df), 2):  # 홀수 행 (0-based index: row 1, row 3, ...)
            id_row = df.iloc[i]  # 아이디가 있는 행

            for col_index in range(len(id_row)):
                if pd.isna(id_row[col_index]):  # 아이디가 없는 셀은 건너뜀
                    continue
                
                if image_file_index >= len(image_files):  # 이미지 파일이 더 이상 없으면 종료
                    print("No more images to process.")
                    return

                id_value = id_row[col_index]  # 아이디 값 추출
                image_filename = f"{id_value}.png"
                image_output_path = os.path.join(output_folder, image_filename)

                with open(image_output_path, 'wb') as f:
                    f.write(z.read(image_files[image_file_index]))

                image_file_index += 1  # 다음 이미지로 이동

# 이미지 추출할 엑셀 파일과 이미지 저장할 폴더 경로
excel_file = './data/raw/CO_IMAGE_DSS_반출.xlsx'
output_folder = './data/image'

# 이미지 추출 함수 실행
extract_images_with_ids(excel_file, output_folder)